# Read CSV

In [6]:
import csv
from functools import reduce

def read_csv(file_name):
    with open(file_name, "r") as f:
        pcb_names = list(csv.reader(f, delimiter=","))

    pcb_names = reduce(lambda xs, ys: xs + ys, pcb_names)
    return pcb_names

# Deal with fp_text

In [7]:
from collections import defaultdict

def revise_module_fp_text(pcb):

    text_count = defaultdict(int)
    for m in pcb.module:
        text_count[m.fp_text[0][1]] += 1

    print(text_count)
    for m in pcb.module:
        if text_count[m.fp_text[0][1]] > 1 or m.fp_text[0][1] == '""' or m.fp_text[0][1] == '" "':
            tmp_text = m.fp_text[0][1]
            if tmp_text == '""':
                m.fp_text[0][1] =  f"Noname_{text_count[tmp_text]}"
            elif tmp_text == '" "':
                m.fp_text[0][1] =  f"None_{text_count[tmp_text]}"
            else:
                m.fp_text[0][1] =  str(tmp_text) + "_" + str(text_count[tmp_text])
            del m.fp_text[0][1][0]
            text_count[tmp_text] -= 1

# Deal with fill zones

In [8]:
import sys
sys.path.append('..')
from Data_extraction.thirdparty.kicad_parser.kicad_pcb import *

import os

def delete_segment_via(pcb, delete_nets):

    seg_idx = []
    via_idx = []
    for i in range(len(pcb.segment)):
        if pcb.segment[i].net in delete_nets:
            seg_idx.append(i)
    seg_idx.sort()
    for i in range(len(pcb.via)):
        if pcb.via[i].net in delete_nets:
            via_idx.append(i)
    via_idx.sort()
    # print(seg_idx, via_idx)

    bc = 0
    for i in seg_idx:
        del_i = i - bc
        if pcb.segment[del_i].net in delete_nets:
            del pcb.segment[del_i]
            bc += 1
        else:
            print(f"{i} is not in the delete net for segment")

    bc = 0
    for i in via_idx:
        del_i = i - bc
        if pcb.via[del_i].net in delete_nets:
            del pcb.via[del_i]
            bc += 1
        else:
            print(f"{i} is not in the delete net for via")


# pcb_names = read_csv("good_raw.csv")

# print(len(pcb_names), pcb_names)

# fill_zone_folder = "good_raws"
# if not os.path.exists(fill_zone_folder):
#     os.mkdir(fill_zone_folder)

# for name in pcb_names:
#     pcb_file_path = "../../PCBs_new/" + name + "/raw.kicad_pcb"
#     new_pcb_name = name + ".kicad_pcb"
#     pcb = KicadPCB.load(pcb_file_path)
#     fill_zone_nets = set([])
#     for zone in pcb.zone:
#         fill_zone_nets.add(zone.net)
#     print(name)
#     del pcb.zone
#     delete_segment_via(pcb, fill_zone_nets)
#     revise_module_fp_text(pcb)
#     pcb.export(os.path.join(fill_zone_folder, new_pcb_name))



# Check FreeRouting results

In [2]:
import os

pcb_names = read_csv("route_fill_zones.csv")
results_folder = "./deleted_fill_zones/"
resuts_files = list(os.listdir(results_folder))
for pcb_name in pcb_names:
    res_name = pcb_name + ".ses"
    if res_name not in resuts_files:
        print(pcb_name)
    

kitspace_d20_r0.1
kitspace_d20_tri_r0.2
kitspace_d20_tri_r0.3
kitspace_EEZ%20DIB%20MCU%20r1B2
kitspace_glasgow
kitspace_lpa96
kitspace_mppt-1210-hus
kitspace_OtterCastAmp
kitspace_tri_panel_4up_r0.1
kitspace_ulx3s
kitspace_ulx4m
kitspace_USB-C-Screen-Adapter
LibreSolar_MPPT_charger_20A


# Update PCBs folder

Put processed.kicad_pcb into folder

In [4]:
import os
import shutil

good_raw = read_csv("./good_raw.csv")
fill_zones_pcbs = read_csv("./route_fill_zones.csv")
print(good_raw)
print(fill_zones_pcbs)

pcb_folder = '../../PCBs/'
for pcb in os.listdir(pcb_folder):
    target = '../../PCBs/' + pcb + '/processed.kicad_pcb'
    if pcb in good_raw:
        source = './good_raws/' + pcb + '.kicad_pcb'
        shutil.copy(source, target)
    elif pcb in fill_zones_pcbs:
        source = './deleted_fill_zones/' + pcb + ".kicad_pcb"
        shutil.copy(source, target)
    elif os.path.exists(target):
        os.remove(target)

    if os.path.exists('../../PCBs/' + pcb + '/error.log'):
        os.remove('../../PCBs/' + pcb + '/error.log')

    if os.path.exists('../../PCBs/' + pcb + '/final.json'):
        os.remove('../../PCBs/' + pcb + '/final.json')


['kitspace_antenna_analyser', 'bobc_control_panel', 'bobc_matrix_clock', 'bobc_LCD-panel-adapter-lvc', 'bobc_mbeduinopresso', 'bobc_MS-F100', 'cnlohr_wiflier', 'kitspace_bbq10breakout', 'kitspace_BQ25570_Harvester', 'kitspace_Brk40p', 'kitspace_CA3306E', 'kitspace_CH330', 'kitspace_ChaosLooper', 'kitspace_CO2', 'kitspace_CocoMixtape_UGM_Kicad', 'kitspace_cseduinov4', 'kitspace_DIY_detector', 'kitspace_dynamixel_shield', 'kitspace_esp8266', 'kitspace_esp8266-12-breakout', 'kitspace_ESP8266-PowerMonitor', 'kitspace_f-91w', 'kitspace_fast_diode_pcb', 'kitspace_filaSens', 'kitspace_gas_sensor', 'kitspace_GPSMux', 'kitspace_grove_adaptor', 'kitspace_hbridge_driver', 'kitspace_Heltec_Node', 'kitspace_hum_temp_sensor', 'kitspace_ir_sensor', 'kitspace_Mixtape_Barebone', 'kitspace_nunchuk_breakout', 'kitspace_OutlineChaos', 'kitspace_PCBs', 'kitspace_pmt_combiner', 'kitspace_postcard', 'kitspace_Potentiometer_mount_4LED', 'kitspace_Potentiometer_mount_8LED', 'kitspace_Potentiometer_mount_16LED'

In [20]:
import os
import sys
sys.path.append('..')
from Data_extraction.thirdparty.kicad_parser.kicad_pcb import *

fill_zone_folder = "./Cleaning_PCBs/"
pcb_names = ["kitspace_d20_tri_r0.2", "kitspace_d20_tri_r0.3"]
for name in pcb_names:
    pcb_file_path = "../../PCBs/" + name + "/raw.kicad_pcb"
    new_pcb_name = name + ".kicad_pcb"
    pcb = KicadPCB.load(pcb_file_path)
    fill_zone_nets = set([])
    for zone in pcb.zone:
        fill_zone_nets.add(zone.net)
    print(name)
    print(fill_zone_nets)
    del pcb.zone
    delete_segment_via(pcb, fill_zone_nets)
    revise_module_fp_text(pcb)

    for m_i, m in enumerate(pcb.module):
        for p_i, p in enumerate(pcb.module[m_i].pad):
            if "net" in p and p["net"][0] in fill_zone_nets:
                del pcb.module[m_i].pad[p_i]["net"]

    pcb.export(os.path.join(fill_zone_folder, new_pcb_name))



kitspace_d20_tri_r0.2
{2}
defaultdict(<class 'int'>, {'J1': 1, 'D1': 1, 'D90': 1, 'D17': 1, 'D2': 1, 'D3': 1, 'D4': 1, 'D5': 1, 'D6': 1, 'D7': 1, 'D8': 1, 'D9': 1, 'D10': 1, 'D11': 1, 'D12': 1, 'D13': 1, 'D14': 1, 'D15': 1, 'D18': 1, 'D19': 1, 'D20': 1, 'D21': 1, 'D22': 1, 'D23': 1, 'D24': 1, 'D25': 1, 'D26': 1, 'D27': 1, 'D28': 1, 'D29': 1, 'D30': 1, 'D33': 1, 'D34': 1, 'D35': 1, 'D36': 1, 'D37': 1, 'D38': 1, 'D39': 1, 'D40': 1, 'D41': 1, 'D42': 1, 'D43': 1, 'D44': 1, 'D45': 1, 'D49': 1, 'D50': 1, 'D51': 1, 'D52': 1, 'D53': 1, 'D54': 1, 'D55': 1, 'D56': 1, 'D57': 1, 'D58': 1, 'D59': 1, 'D60': 1, 'D65': 1, 'D66': 1, 'D67': 1, 'D68': 1, 'D69': 1, 'D70': 1, 'D71': 1, 'D72': 1, 'D73': 1, 'D74': 1, 'D75': 1, 'D81': 1, 'D82': 1, 'D83': 1, 'D84': 1, 'D85': 1, 'D86': 1, 'D87': 1, 'D88': 1, 'D89': 1, 'D103': 1, 'D97': 1, 'D98': 1, 'D99': 1, 'D100': 1, 'D101': 1, 'D102': 1, 'D104': 1, 'D105': 1, 'D113': 1, 'D114': 1, 'D115': 1, 'D116': 1, 'D117': 1, 'D118': 1, 'D119': 1, 'D120': 1, 'D129': 1, '